XOR Training mit verschlüsselten Daten
---
Nun soll ein XOR Modell trainiert mit verschlüsselten Daten werden. Dafür muss in der Inferenz verwendeten Architektur alle Aktivierungsfunktionen mit der quadratischen Aktivierungsfunktion ersetzt werden.

In [3]:
from util.heNet import Network, FullyConnectedLayer, ActivationLayer
from util.heNet import square, square_prime, binary_cross_entropy, binary_cross_entropy_prime
import numpy as np
import tenseal as ts

poly_mod_degree = 2 ** 13
bits_scale = 40

coeff_mod_bit_sizes=[60, bits_scale, 60]
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)

context.global_scale = pow(2, bits_scale)
context.generate_galois_keys()

In [4]:
from util.heNet import EncryptedActivationLayer, EncryptedFullyConnectedLayer

net = Network(25)
net.add(EncryptedFullyConnectedLayer(2, 2, context))
net.add(EncryptedActivationLayer(square, square_prime, context))
net.add(EncryptedFullyConnectedLayer(2, 1, context))
net.add(EncryptedActivationLayer(square, square_prime, context))

net.use(binary_cross_entropy, binary_cross_entropy_prime)

In [4]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

In [9]:
net.crypt_fit(x_train, y_train, epochs=100, learning_rate=0.1, context=context)

epoch 1/100
epoch 26/100
epoch 51/100
epoch 76/100


In [10]:
out = net.predict(x_train)
print(out)

[array([[0.0645173]]), array([[0.79783051]]), array([[0.81424527]]), array([[0.02962624]])]


Wie man sind die Ausgaben wie erwartet in der Form von 0 1 1 0 (die Ausgaben weichen etwas ab aber wären durch eine Fallunterscheid mit 0.5 als Grenze immer noch richtig) und somit hat das Modell die Abbildung eines XOR Gatters mit den verschlüsselten Daten gelernt.

Iris Training mit verschlüsselten Daten
---
In diesem Test wird der interaktive Ansatz auf dem Iris Datensatz ausprobiert, um ein Modell auf den verschlüsselten Daten zu trainieren das die Klasse der Blumen vorhersagen kann. Hierfür wird wieder der benötigte Datensatz zuerst geladen.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

def transform(data):
    return np.array([[row] for row in data])

X_train = transform(X_train)
X_test = transform(X_test)
y_train = transform(y_train)
y_test = transform(y_test)

Das fürs Training verwendete Modell in der Inferenz wird hier von der Architektur her wieder verwendet. Nur muss wie bereits erwähnt die letzte Aktivierungsfunktion ersetzt werden.

In [29]:
def encrypted_training(context):
    net = Network(debug=5)
    net.add(EncryptedFullyConnectedLayer(4, 4, context))
    net.add(EncryptedActivationLayer(square, square_prime, context))
    net.add(EncryptedFullyConnectedLayer(4, 3, context))
    net.add(EncryptedActivationLayer(square, square_prime, context))

    net.use(binary_cross_entropy, binary_cross_entropy_prime)

    net.crypt_fit(X_train, y_train, epochs=10, learning_rate=0.002, minibatch=8, context=context)

    outN = net.predict(X_test)
    correct = 0
    for idx,_ in enumerate(outN):
        if np.argmax(np.squeeze(outN[idx])) == np.argmax(np.squeeze(y_test[idx])):
            correct = correct + 1
    print(f"Im Testdatensatz wurden {correct} von {len(outN)} richtig klassifiziert --> {round(correct/len(outN),2)}")

Es wird ein kleiner Verschlüsselungskontext gewählt, um schnelle Berechnungen in den Schichten zu ermöglichen.

In [12]:
poly_mod_degree = 2 ** 13
bits_scale = 40

coeff_mod_bit_sizes=[60, bits_scale, 60]
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)

context.global_scale = pow(2, bits_scale)
context.generate_galois_keys()

encrypted_training(context)

epoch 1/10
epoch 6/10
Im Testdatensatz wurden 29 von 30 richtig klassifiziert --> 0.97


Die Genauigkeit dieses Modells ist etwas schlechter als das unverschlüsselte Modell (während der Inferenz), vermutlich weil die Lernrate niedriger ist und es dadurch nicht perfekt optimiert ist. Die Lernrate muss aber kleiner sein, denn ansonsten geraten die Werte durch die quadratischen Aktivierungsfunktionen auser Kontrolle  (beziehungsweise die Gewichte im Netz werden zu groß).